<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 3.5: Object Oriented Programming
**Prev: [Functional Programming](3.4_functional_programming.ipynb)**<br>
**Next: [Types](3.6_types.ipynb)**

## Motivation
Scala 和 Chisel 是面向对象的编程语言，这意味着代码可以被分隔到对象中。
Scala 建立在 Java 之上，继承了许多 Java 的面向对象特性。
然而，正如我们将在下面看到的，有一些不同之处。
Chisel 的硬件模块类似于 Verilog 的模块，因为它们可以被实例化并作为单个或多个实例连接起来。

## Setup

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.experimental._
import chisel3.tester._
import chisel3.tester.RawTester.test

---
# Object Oriented Programming
本节概述了 Scala 如何实现面向对象的编程范式。到目前为止，你已经看到了类，但 Scala 还具有以下特性：
- [Abstract Classes](#abstract)
- [Traits](#traits)
- [Objects](#objects)
- [Companion Objects](#compobj)
- [Case Classes](#caseclass)

## Abstract Classes<a name="abstract"></a>
Abstract Classes 就像其他编程语言的实现一样。它们可以定义许多未实现的值，子类必须实现这些值。任何对象只能直接继承一个父 Abstract Class。

<span style="color:blue">**示例：Abstract Class**</span><br>

In [ ]:
abstract class MyAbstractClass {
  def myFunction(i: Int): Int
  val myValue: String
}
class ConcreteClass extends MyAbstractClass {
  def myFunction(i: Int): Int = i + 1
  val myValue = "Hello World!"
}
// Uncomment below to test!
// val abstractClass = new MyAbstractClass() // Illegal! Cannot instantiate an abstract class
val concreteClass = new ConcreteClass()      // Legal!


## Traits<a name="traits"></a>
Traits 与 Abstract Classes 非常相似，它们可以定义未实现的值。然而，它们有两个不同之处：
- 一个类可以继承多个 Traits
- 一个 Trait 不能有构造函数参数

<span style="color:blue">**示例：Traits 和多重继承**</span><br>
Traits 是 Scala 实现多重继承的方式，如下面的示例所示。`MyClass` 继承了两个 Traits：`HasFunction` 和 `HasValue`：

In [ ]:
trait HasFunction {
  def myFunction(i: Int): Int
}
trait HasValue {
  val myValue: String
  val myOtherValue = 100
}
class MyClass extends HasFunction with HasValue {
  override def myFunction(i: Int): Int = i + 1
  val myValue = "Hello World!"
}
// Uncomment below to test!
// val myTraitFunction = new HasFunction() // Illegal! Cannot instantiate a trait
// val myTraitValue = new HasValue()       // Illegal! Cannot instantiate a trait
val myClass = new MyClass()                // Legal!

要继承多个 Traits，可以像这样链接它们：

```scala
class MyClass extends HasTrait1 with HasTrait2 with HasTrait3 ...
```

一般来说，除非你确定要强制执行 Abstract Classes 的单一继承限制，否则始终使用 Traits。

## Objects<a name="objects"></a>
Scala 有一个针对这些单例类的语言特性，称为 Objects。你不能实例化一个 Object **（不需要调用 `new`）**；你可以直接引用它们。这使得它们类似于 Java 的静态类。

<span style="color:blue">**示例：Objects**</span><br>

In [ ]:
object MyObject {
  def hi: String = "Hello World!"
  def apply(msg: String) = msg
}
println(MyObject.hi)
println(MyObject("This message is important!")) // equivalent to MyObject.apply(msg)

## Companion Objects<a name="compobj"></a>

当一个类（class）和一个对象（object）共享相同的名称并定义在同一个文件中时，该对象称为**伴生对象**。当你在类/对象名称前使用 `new` 时，它将实例化该类。如果你不使用 `new`，它将引用该对象：

<span style="color:blue">**示例：Companion Object**</span><br>

In [ ]:
object Lion {
    def roar(): Unit = println("I'M AN OBJECT!")
}
class Lion {
    def roar(): Unit = println("I'M A CLASS!")
}
new Lion().roar()
Lion.roar()

伴生对象通常用于以下原因：
1. 包含与类相关的常量
2. 在类构造函数之前/之后执行代码
3. 为类创建多个构造函数

在下面的示例中，我们将实例化多个 Animal 实例。我们希望每个动物都有一个名字，并且知道它在所有实例化中的顺序。最后，如果没有给定名字，它应该得到一个默认名字。

In [ ]:
object Animal {
    val defaultName = "Bigfoot"
    private var numberOfAnimals = 0
    def apply(name: String): Animal = {
        numberOfAnimals += 1
        new Animal(name, numberOfAnimals)
    }
    def apply(): Animal = apply(defaultName)
}
class Animal(name: String, order: Int) {
  def info: String = s"Hi my name is $name, and I'm $order in line!"
}

val bunny = Animal.apply("Hopper") // Calls the Animal factory method
println(bunny.info)
val cat = Animal("Whiskers")       // Calls the Animal factory method
println(cat.info)
val yeti = Animal()                // Calls the Animal factory method
println(yeti.info)


**这里发生了什么？**
1. 我们的**Animal 伴生对象**定义了一个与```class Animal```相关的常量：
```scala
val defaultName = "Bigfoot"
```
2. 它还定义了一个私有的可变整数来跟踪 Animal 实例的顺序：
```scala
private var numberOfAnimals = 0
```
3. 它定义了两个**apply**方法，这些方法被称为**工厂方法**，因为它们返回**class Animal**的实例。
    1. 第一个使用一个参数```name```创建一个 Animal 实例，并使用```numberOfAnimals```来调用 Animal 类构造函数。
```scala
def apply(name: String): Animal = {
            numberOfAnimals += 1
            new Animal(name, numberOfAnimals)
}
```
    2. 第二个工厂方法不需要参数，而是使用默认名称来调用另一个 apply 方法。
```scala
def apply(): Animal = apply(defaultName)
```
4. 可以像这样简单地调用这些工厂方法
```scala
val bunny = Animal.apply("Hopper")
```
这消除了使用 new 关键字的需要，但真正的魔力在于编译器假定任何时候它看到括号应用于实例（instance）或对象（object）时调用 apply 方法：
```scala
val cat = Animal("Whiskers")
```
5. 工厂方法通常通过伴生对象提供，允许以替代方式表达实例创建，提供对构造函数参数的附加测试、转换，并消除了使用```new```关键字的需要。请注意，必须调用伴生对象的 `apply` 方法，`numberOfAnimals` 才会递增。

**Chisel 使用了许多伴生对象，如 Module。** 当你写以下内容时：
```scala
val myModule = Module(new MyModule)
```
你正在调用**Module 伴生对象**，因此 Chisel 可以在实例化```MyModule```之前和之后运行后台代码。

## Case Classes<a name="caseclass"></a>
Case Classes 是一种特殊类型的 Scala 类，提供了一些很酷的附加功能。它们在 Scala 编程中非常常见，本节概述了一些有用的功能：
- 允许**外部访问** **类参数**
- **消除**实例化类时使用 **`new`** 的需要
- 自动创建一个 **unapply 方法**，提供对所有类参数的访问
- 不能被子类化

在下面的示例中，我们声明了三种不同的类，`Nail`、`Screw` 和 `Staple`。

In [ ]:
class Nail(length: Int) // Regular class
val nail = new Nail(10) // Requires the `new` keyword
// println(nail.length) // Illegal! Class constructor parameters are not by default externally visible

class Screw(val threadSpace: Int) // By using the `val` keyword, threadSpace is now externally visible
val screw = new Screw(2)          // Requires the `new` keyword
println(screw.threadSpace)

case class Staple(isClosed: Boolean) // Case class constructor parameters are, by default, externally visible
val staple = Staple(false)           // No `new` keyword required
println(staple.isClosed)

`Nail` 是一个普通的类，因为我们没有在参数列表中使用 `val` 关键字，所以它的参数在外部不可见。声明 `Nail` 的实例时还需要使用 `new` 关键字。

`Screw` 的声明类似于 `Nail`，但在参数列表中包含 `val`。这使得它的参数 `threadSpace` 在外部可见。

通过使用 case class，`Staple` 获得了所有参数都在外部可见的好处（无需使用 `val` 关键字）。

此外，声明 case class 时，`Staple` 不需要使用 `new` 关键字。这是因为 Scala 编译器会自动为代码中的每个 case class 创建一个伴生对象，该对象包含一个 apply 方法用于 case class。

Case Classes 是参数众多的生成器的良好容器。构造函数为定义派生参数和验证输入提供了一个好地方。

In [ ]:
case class SomeGeneratorParameters(
    someWidth: Int,
    someOtherWidth: Int = 10,
    pipelineMe: Boolean = false
) {
    require(someWidth >= 0)
    require(someOtherWidth >= 0)
    val totalWidth = someWidth + someOtherWidth
}

---
# Inheritance with Chisel<a name="super"></a>
你已经见过 `Module` 和 `Bundle`，但了解它们的实际运作方式非常重要。
你创建的每个 Chisel 模块都是一个扩展了基础类型 `Module` 的类。
你创建的每个 Chisel IO 都是一个扩展了基础类型 `Bundle` 的类（或者在某些特殊情况下，扩展了 `Bundle` 的超类型 [`Record`](https://github.com/freechipsproject/chisel3/blob/v3.0.0/chiselFrontend/src/main/scala/chisel3/core/Aggregate.scala#L415)）。
像 `UInt` 或 `Bundle` 这样的 Chisel 硬件类型都以 `Data` 作为超类型。
我们将探讨使用面向对象编程来创建层次化硬件块并探讨对象重用。在下一模块关于类型泛型生成器中，你将更多地了解类型和 `Data`。

## Module<a name="module"></a>
每当你想在 Chisel 中创建一个硬件对象时，它都需要以 `Module` 作为超类。
继承并不总是重用的正确工具（[组合（composition）优于继承（inheritance）](https://en.wikipedia.org/wiki/Composition_over_inheritance) 是一个常见原则），但继承仍然是一个强大的工具。
下面是创建一个 `Module` 并层次化地连接多个实例的示例。

<span style="color:blue">**示例：Gray 编码器和解码器**</span><br>
我们将创建一个硬件 Gray 编码器/解码器。编码或解码操作的选择是硬件可编程的。

In [ ]:
import scala.math.pow

// create a module
class GrayCoder(bitwidth: Int) extends Module {
  val io = IO(new Bundle{
    val in = Input(UInt(bitwidth.W))
    val out = Output(UInt(bitwidth.W))
    val encode = Input(Bool()) // decode on false
  })
  
  when (io.encode) { //encode
    io.out := io.in ^ (io.in >> 1.U)
  } .otherwise { // decode, much more complicated
    io.out := Seq.fill(log2Ceil(bitwidth))(Wire(UInt(bitwidth.W))).zipWithIndex.fold((io.in, 0)){
      case ((w1: UInt, i1: Int), (w2: UInt, i2: Int)) => {
        w2 := w1 ^ (w1 >> pow(2, log2Ceil(bitwidth)-i2-1).toInt.U)
        (w2, i1)
      }
    }._1  // scala中，._1 用于获取一个元组（tuple）中的第一个元素
  }
}


试试看吧！

In [ ]:
// test our gray coder
val bitwidth = 4
test(new GrayCoder(bitwidth)) { c =>
    def toBinary(i: Int, digits: Int = 8) = {
        String.format("%" + digits + "s", i.toBinaryString).replace(' ', '0')
    }
    println("Encoding:")
    for (i <- 0 until pow(2, bitwidth).toInt) {
        c.io.in.poke(i.U)
        c.io.encode.poke(true.B)
        c.clock.step(1)
        println(s"In = ${toBinary(i, bitwidth)}, Out = ${toBinary(c.io.out.peek().litValue.toInt, bitwidth)}")
    }

    println("Decoding:")
    for (i <- 0 until pow(2, bitwidth).toInt) {
        c.io.in.poke(i.U)
        c.io.encode.poke(false.B)
        c.clock.step(1)
        println(s"In = ${toBinary(i, bitwidth)}, Out = ${toBinary(c.io.out.peek().litValue.toInt, bitwidth)}")
    }

}

Gray 码通常用于异步接口。通常使用 Gray 计数器而不是功能齐全的编码器/解码器，但我们将使用上面的模块来简化操作。下面是一个使用上述 Gray 编码器构建的 AsyncFIFO 示例。控制逻辑和测试器留作稍后的练习。现在，看看 Gray 编码器是如何多次实例化和连接的。

In [ ]:
class AsyncFIFO(depth: Int = 16) extends Module {
  val io = IO(new Bundle{
    // write inputs
    val write_clock = Input(Clock())
    val write_enable = Input(Bool())
    val write_data = Input(UInt(32.W))
    
    // read inputs/outputs
    val read_clock = Input(Clock())
    val read_enable = Input(Bool())
    val read_data = Output(UInt(32.W))
    
    // FIFO status
    val full = Output(Bool())
    val empty = Output(Bool())
  })
  
  // add extra bit to counter to check for fully/empty status
  assert(isPow2(depth), "AsyncFIFO needs a power-of-two depth!")
  val write_counter = withClock(io.write_clock) { Counter(io.write_enable && !io.full, depth*2)._1 }
  val read_counter = withClock(io.read_clock) { Counter(io.read_enable && !io.empty, depth*2)._1 }
  
  // encode
  val encoder = new GrayCoder(write_counter.getWidth)
  encoder.io.in := write_counter
  encoder.io.encode := true.B
  
  // synchronize
  val sync = withClock(io.read_clock) { ShiftRegister(encoder.io.out, 2) }
  
  // decode
  val decoder = new GrayCoder(read_counter.getWidth)
  decoder.io.in := sync
  decoder.io.encode := false.B
  
  // status logic goes here
  
}

---
# You're done!

[Return to the top.](#top)